# Secondo prototipo di modello ML: random forest

Liberie varie da installare

In [ ]:
#!pip install pandas
#!pip install sklearn
#!pip install seaborn
#!pip install imblearn
#!pip install shap -U

Inclusione delle librerie utilizzate

In [ ]:
from os.path import exists
from datetime import date
import time
import re
import glob
import shap
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA, IncrementalPCA, SparsePCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

Variabili di gestione files, parametri del modello e della fase di training

In [ ]:
######################################## INPUT DATASET ###########################################
# Path of the dataset in .pkl format, can be changed
PATH_DATASET = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_output\filtered_active_bankruptcy_raw_full_2_CE.pkl"
##################################################################################################

######################################## DATA STANDARDIZATION ####################################
# True = Standardize data, can be changed
to_standardize = True
##################################################################################################

######################################## DATA CLASSES BALANCE ####################################
# True = Oversample the least populated class (Bankruptcy), can be changed
avoid_imbalanced_training = False

# Oversample/Undersample the least/most populated class (Bankruptcy), possible values: 'Oversample'
# or 'Undersample'
# It only affects the notebook if avoid_imbalanced_training is True
imbalanced_data_technique = "Oversample"
##################################################################################################

######################################## DIMENSIONALITY REDUCTION ################################
# True = use a dimensionality reduction method, can be changed
dimensionality_reduction = False

# Applies dimensionality reduction method, possible values: 'PCA', 'LDA', 'Incremental PCA',
# 'Sparse PCA'
# It only affects the notebook if dimensionality_reduction is True
dimensionality_reduction_method = "PCA"

# Number of desired components after applying a dimensionality reduction method
# N.B. n_components must be < number of input features and LDA only supports 1 components in
# our case
n_components = 50
##################################################################################################

######################################## EXTRA DATASET PARAMETERS ################################
# True = Also use non financial indexes features like the legal form or the size of the company
additional_features = False
##################################################################################################

######################################## TRAIN-TEST SPLIT ########################################
# A value between [0, 1], it represent the percentage of records not used during training time,
# can be changed
train_test_split_amount = 0.25
##################################################################################################

######################################## EXTRA MODEL PARAMETERS ##################################
# Select a random state value in order to control the randomness effect, can be changed
rnd_state = 25

# Specify the number of cuncurrent jobs in order to speed up certain traning phases.
# Specify -1 in order to use all the job available, the default one is 1, can be changed
n_jobs = -1

# Select the number of tree used by the random forest
# It should be something between [10, 200], default value is 100
n_tree = 100
##################################################################################################

######################################## SHAP VALUES PARAMETERS ##################################
# True = Print shap values plot
shap_plot = False
##################################################################################################

######################################## OUTPUT EXPERIMENT #######################################
# True = Export the current experiment inside the dataframe that collects all of them, can be changed
to_export = True

# Path of the dataset in .pkl format to store all the experiments, can be changed and
# if not present it will be created
OUTPUT_PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\ML_models"

# True = Export the trained model on an external file, can be changed
to_export_model = False

# Path of the exported model, can be changed
OUTPUT_PATH_MODEL = ""

# True = Export the current classification metrics for each region if available, can be changed
to_export_region_metrics = False

# Path of the exported regional metrics dataframe as xlsx file, can be changed
OUTPUT_REGION_PATH = ""
##################################################################################################

Lettura del dataset di input

In [ ]:
dataset = pd.read_pickle(PATH_DATASET)
print("Il dataset da utilizzare ha", dataset.shape[0], "record e", dataset.shape[1], "colonne")

Suddivisione del dataset in X e Y, dove X sono le features in ingresso e Y è la risposta in output (attivo/bancarotta)

In [ ]:
# Remove descriptive columns
removed_columns = [
    "Ragione sociale",
    "Province",
    "Accounting closing date",
    "Legal Status",
    "Tax code number",
    "CCIAA number"
]

# List to store region names
region_names = []



if not additional_features:
    removed_columns.append("Legal Form")
    removed_columns.append("Company Size")
    removed_columns.append("Number of employees")
    removed_columns.append("Regione")

# X dataset
X_dataset = dataset.copy()
X_dataset.drop(removed_columns, axis=1, inplace=True, errors="ignore")

# Columns to be standardized
to_be_std_columns = list(X_dataset.columns)

# Manage additional features
if additional_features:
    # One hot encoding
    if "Legal Form" in X_dataset.columns:
        X_dataset = X_dataset.join(pd.get_dummies(dataset['Legal Form']))
        X_dataset.drop('Legal Form', axis=1, inplace=True)
        to_be_std_columns.remove('Legal Form')


    if "Company Size" in X_dataset.columns:
        X_dataset = X_dataset.join(pd.get_dummies(dataset['Company Size']))
        X_dataset.drop('Company Size', axis=1, inplace=True)
        to_be_std_columns.remove('Company Size')

    if "Regione" in X_dataset.columns:
        region_names = list(dataset['Regione'].unique())
        X_dataset = X_dataset.join(pd.get_dummies(dataset['Regione']))
        X_dataset.drop('Regione', axis=1, inplace=True)
        to_be_std_columns.remove('Regione')


# Save the new feature names
X_features_names = X_dataset.columns.to_list()

# Y dataset
Y_dataset = dataset['Legal Status'].copy()

Stampo i primi record dei 2 nuovi dataset per chiarezza

In [ ]:
X_dataset.head()

In [ ]:
Y_dataset.head()

Codifico la variabile di risposta (Active/Bankruptcy) in (0/1)

In [ ]:
Y_dataset.replace({"Active": 0, "Bankruptcy": 1}, inplace=True)
Y_dataset.head()

Controllo il numero di record per ciascuna classe

In [ ]:
Y_dataset.value_counts()

Uso la tecnica di random oversampling o undersampling per evitare un allenamento di un modello con classi sbilanciate

In [ ]:
if avoid_imbalanced_training:
    # Oversample (provare anche per undersampling)
    if imbalanced_data_technique == "Oversample":
        sm = SMOTE(random_state=rnd_state, n_jobs=n_jobs)
        X_dataset, Y_dataset = sm.fit_resample(X_dataset, Y_dataset)
    # Undersample
    elif imbalanced_data_technique == "Undersample":
        undersample = RandomUnderSampler(sampling_strategy='majority', random_state=rnd_state)
        X_dataset, Y_dataset = undersample.fit_resample(X_dataset, Y_dataset)
    else:
        print("Error: wrong variable value about imbalanced data")
Y_dataset.value_counts()

Applico algoritmo di riduzione della dimensionalità

In [ ]:
if dimensionality_reduction:
    if dimensionality_reduction_method == "PCA":
        dimensionality_reduction_model = PCA(n_components=n_components)
        X_dataset = dimensionality_reduction_model.fit_transform(X_dataset)
    elif dimensionality_reduction_method == "Incremental PCA":
        dimensionality_reduction_model = IncrementalPCA(n_components=n_components)
        X_dataset = dimensionality_reduction_model.fit_transform(X_dataset)
    elif dimensionality_reduction_method == "Sparse PCA":
        dimensionality_reduction_model = SparsePCA(n_components=n_components)
        X_dataset = dimensionality_reduction_model.fit_transform(X_dataset)
    elif dimensionality_reduction_method == "LDA":
        n_components = 1
        dimensionality_reduction_model = LinearDiscriminantAnalysis(n_components=n_components)
        X_dataset = dimensionality_reduction_model.fit_transform(X_dataset, Y_dataset)
    # Need to change feature names, because they're lost in the new dataset
    X_features_names = ["Component " + str(i) for i in range(0, n_components)]

Standardizzo i dati contenuti in X

In [ ]:
if to_standardize:
    # Standardize only the features that are not one hot encoded
    scaler = ColumnTransformer([
        ('STD', StandardScaler(), to_be_std_columns)
    ], remainder='passthrough')
    X_dataset = pd.DataFrame(scaler.fit_transform(X_dataset))
    X_dataset.columns = X_features_names

Esporto il modello che normalizza i dati su un file esterno

In [ ]:
model_number = len(glob.glob1(OUTPUT_PATH_MODEL,"random_forest*.sav"))
scaler_number = len(glob.glob1(OUTPUT_PATH_MODEL,"standard_scaler*.sav"))
if to_export_model:
    # Create the model filename
    scaler_filename = ""
    # Model type
    scaler_filename += "standard_scaler"
    # Enumerate model of the same type
    scaler_filename += "_" + str(max(model_number, scaler_number))
    # File extension
    scaler_filename += ".sav"
    # Export the model
    joblib.dump(scaler, OUTPUT_PATH_MODEL + "/" + scaler_filename)
    print("Scaler esportato in:", OUTPUT_PATH_MODEL + "/" + scaler_filename)

Divido i 2 dataset in train e test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_dataset,
                                                    Y_dataset,
                                                    stratify=Y_dataset,
                                                    test_size=train_test_split_amount,
                                                    random_state=rnd_state)

Creo primo prototipo di random forest e lo alleno su una porzione dei dati

In [ ]:
random_forest_classifier = RandomForestClassifier(random_state=rnd_state, n_jobs=n_jobs, n_estimators=n_tree)
# Add feature names as attribute in order to retrieve them when exported
random_forest_classifier.feature_names = X_features_names
start = time.time()
random_forest_classifier.fit(X_train, Y_train)
stop = time.time()
training_time = stop - start

Esporto il modello su un file esterno

In [ ]:
if to_export_model:
    # Create the model filename
    model_filename = ""
    # Model type
    model_filename += "random_forest"
    # Enumerate model of the same type
    model_filename += "_" + str(max(model_number, scaler_number))
    # File extension
    model_filename += ".sav"
    # Export the model
    joblib.dump(random_forest_classifier, OUTPUT_PATH_MODEL + "/" + model_filename)
    print("Modello esportato in:", OUTPUT_PATH_MODEL + "/" + model_filename)

Analizzo il comportamento del modello sui dati di test che non ha mai visto

In [ ]:
Y_predicted = random_forest_classifier.predict(X_test)
score = accuracy_score(Y_test, Y_predicted)

print("L'accuratezza è", score)

Curva ROC

In [ ]:
probs = random_forest_classifier.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(Y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Matrice di confusione

In [ ]:
conf_matrix = confusion_matrix(y_true=Y_test, y_pred=Y_predicted)
conf_matrix = conf_matrix / conf_matrix.astype(np.float64).sum(axis=1)

ax = plt.subplot()
sns.heatmap(conf_matrix, annot=True, vmin=0.0, vmax=1.0, fmt=".2f", cmap="Blues", ax=ax)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Active', 'Bankruptcy'])
ax.yaxis.set_ticklabels(['Active', 'Bankruptcy'])

Mostro l'importanza di ogni feature

In [ ]:
plt.figure(figsize=(10,50))
feat_importances = pd.Series(random_forest_classifier.feature_importances_, index=X_features_names).sort_values(ascending=False)
feat_importances.plot(kind='barh')

Analisi geografica (se possibile)

In [ ]:
if region_names:
    region_accuracies = []
    region_specificities = []
    region_precisions = []
    region_recalls = []
    region_f1s = []
    region_aucs = []

    # For each region compute classification metrics
    for region_name in region_names[:20]:
        region_Y_test = Y_test[X_test[region_name] == 1]
        region_Y_predicted = random_forest_classifier.predict(X_test[X_test[region_name] == 1])

        region_accuracies.append(accuracy_score(region_Y_test, region_Y_predicted))
        region_specificities.append(recall_score(region_Y_test, region_Y_predicted, pos_label=0))
        region_precisions.append(precision_score(region_Y_test, region_Y_predicted))
        region_recalls.append(recall_score(region_Y_test, region_Y_predicted))
        region_f1s.append(f1_score(region_Y_test, region_Y_predicted))

        region_probs = random_forest_classifier.predict_proba(X_test[X_test[region_name] == 1])
        region_preds = region_probs[:,1]
        region_fpr, region_tpr, region_threshold = metrics.roc_curve(region_Y_test, region_preds)
        region_aucs.append(metrics.auc(region_fpr, region_tpr))


    # Aggregate results into a single dataframe

    region_metrics = pd.DataFrame()

    region_metrics["Accuracy"] = region_accuracies
    region_metrics["Specificity"] = region_specificities
    region_metrics["Precision"] = region_precisions
    region_metrics["Recall"] = region_recalls
    region_metrics["F1-score"] = region_f1s
    region_metrics["AUC"] = region_aucs

    region_metrics.index = region_names[:20]
    region_metrics

In [ ]:
if to_export_region_metrics and region_names:
    export_name = "RF_"
    if avoid_imbalanced_training:
        export_name += (imbalanced_data_technique + "_")
    else:
        export_name += ("Unbalanced" + "_")
    export_name += PATH_DATASET.split("\\")[-1]

    print("Metriche di classificazione esportate in", OUTPUT_REGION_PATH + "/" + export_name + ".xlsx")
    region_metrics.to_excel(OUTPUT_REGION_PATH + "/" + export_name + ".xlsx")

Stampo gli SHAP value inerenti al modello

In [ ]:
if shap_plot:
    explainer = shap.TreeExplainer(random_forest_classifier)
    shap_values = explainer.shap_values(X_test, approximate=True)
    shap.summary_plot(shap_values, X_test, feature_names=X_features_names)

Prelevo i dati non utilizzati per testare il modello

In [ ]:
# Take the complete dataset PATH
path_complete_dataset = re.sub("filtered", "complete", PATH_DATASET)
path_complete_dataset = re.sub("_[0-9]{1,2}\.pkl$", ".pkl", path_complete_dataset)
path_complete_dataset = re.sub("_[0-9]{1,2}_region\.pkl$", "_region.pkl", path_complete_dataset)

# Read the complete dataset
complete_dataset = pd.read_pickle(path_complete_dataset)
print("Lettura dataset:", path_complete_dataset)

In [ ]:
# Take only the record not used during the previous phase
validation_dataset = complete_dataset[~(complete_dataset['Ragione sociale'].isin(dataset['Ragione sociale'])
                                        & complete_dataset['Province'].isin(dataset['Province'])
                                        & complete_dataset['Accounting closing date'].isin(dataset['Accounting closing date'])
                                        )]

In [ ]:
# X validation dataset
X_validation = validation_dataset.copy()
X_validation.drop(removed_columns, axis=1, inplace=True, errors="ignore")

# Y validation dataset
Y_validation = validation_dataset['Legal Status'].copy()
Y_validation.replace({"Active": 0, "Bankruptcy": 1}, inplace=True)

# Manage additional features
if additional_features:
    # One hot encoding

    if "Legal Form" in X_validation.columns:
        X_validation = X_validation.join(pd.get_dummies(validation_dataset['Legal Form']))
        X_validation.drop('Legal Form', axis=1, inplace=True)

    if "Company Size" in X_validation.columns:
        X_validation = X_validation.join(pd.get_dummies(validation_dataset['Company Size']))
        X_validation.drop('Company Size', axis=1, inplace=True)

    if "Regione" in X_validation.columns:
        X_validation = X_validation.join(pd.get_dummies(validation_dataset['Regione']))
        X_validation.drop('Regione', axis=1, inplace=True)

# Apply dimensionality reduction method
if dimensionality_reduction:
    X_validation = dimensionality_reduction_model.transform(X_validation)

# Scale the X validation dataset
if to_standardize:
    # Standardize only the features without that are not one hot encoded
    X_validation = pd.DataFrame(scaler.transform(X_validation))
    X_validation.columns = X_features_names

Distribuzione dei dati non visti dal modello

In [ ]:
Y_validation.value_counts()

Test del modello sui dati del dataset completo non utilizzati

In [ ]:
Y_validation_predicted = random_forest_classifier.predict(X_validation)
score = accuracy_score(Y_validation, Y_validation_predicted)

print("L'accuratezza è", score)

In [ ]:
probs = random_forest_classifier.predict_proba(X_validation)
Y_validation_preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(Y_validation, Y_validation_preds)
roc_auc_validation = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_validation)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
conf_matrix = confusion_matrix(y_true=Y_validation, y_pred=Y_validation_predicted)

ax = plt.subplot()
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt='g', ax=ax)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Active', 'Bankruptcy'])
ax.yaxis.set_ticklabels(['Active', 'Bankruptcy'])

Controllo se esiste il dataset di output

In [ ]:
# Only if we want to export
if to_export:
    if exists(OUTPUT_PATH + "/ML_model_experiments.pkl"):
        # If the dataset exists, read it
        output_dataset = pd.read_pickle(OUTPUT_PATH + "/ML_model_experiments.pkl")
        print("Dataset di output trovato")
    else:
        # Otherwise create it
        output_dataset = pd.DataFrame()
        print("Dataset di output non trovato, creo un nuovo dataset")

Esporto l'esperimento nel dataset di output

In [ ]:
if to_export:
    if not avoid_imbalanced_training:
        imbalanced_data_technique = "N.A."

    if not dimensionality_reduction:
        dimensionality_reduction_method = "N.A."

    # Compute metrics to be added to the dataset
    accuracy = accuracy_score(Y_test, Y_predicted)
    specificity = recall_score(Y_test, Y_predicted, pos_label=0)
    precision = precision_score(Y_test, Y_predicted)
    recall = recall_score(Y_test, Y_predicted)
    f1 = f1_score(Y_test, Y_predicted)
    mcc = matthews_corrcoef(Y_test, Y_predicted)

    accuracy_validation = accuracy_score(Y_validation, Y_validation_predicted)
    specificity_validation = recall_score(Y_validation, Y_validation_predicted, pos_label=0)
    precision_validation = precision_score(Y_validation, Y_validation_predicted)
    recall_validation = recall_score(Y_validation, Y_validation_predicted)
    f1_validation = f1_score(Y_validation, Y_validation_predicted)
    mcc_validation = matthews_corrcoef(Y_validation, Y_validation_predicted)

    tn, fp, fn, tp = confusion_matrix(Y_test, Y_predicted).ravel()
    tn_validation, fp_validation, fn_validation, tp_validation = confusion_matrix(Y_validation, Y_validation_predicted).ravel()


    # Create a new record
    new_record = pd.DataFrame({"Date": [date.today()],
                               "Model type": ["Random Forest"],
                               "Model parameters": [random_forest_classifier.get_params()],
                               "Dataset - train/test": [PATH_DATASET.split("\\")[-1]],
                               "Features": [X_features_names],
                               "Standardized": [to_standardize],
                               "Dimensionality reduction": [dimensionality_reduction],
                               "Dimensionality reduction technique": [dimensionality_reduction_method],
                               "Number of features": [len(X_features_names)],
                               "Features importance": [feat_importances.to_dict()],
                               "Imbalanced data corrections": [avoid_imbalanced_training],
                               "Imbalanced data technique": [imbalanced_data_technique],
                               "Number of active companies - train/test": [Y_dataset.value_counts()[0]],
                               "Number of bankruptcy companies - train/test": [Y_dataset.value_counts()[1]],
                               "Train/Test split": [train_test_split_amount],
                               "Training time": [training_time],
                               "Random state": [rnd_state],
                               "TN - test": [tn],
                               "TP - test": [tp],
                               "FN - test": [fn],
                               "FP - test": [fp],
                               "Accuracy - test": [accuracy],
                               "Specificity - test": [specificity],
                               "Precision - test": [precision],
                               "Recall - test": [recall],
                               "F1-score - test": [f1],
                               "MCC - test": [mcc],
                               "AUC - test": [roc_auc],
                               "Validation dataset": [path_complete_dataset.split("\\")[-1]],
                               "Number of active companies - validation": Y_validation.value_counts()[0],
                               "Number of bankruptcy companies - validation": [Y_validation.value_counts()[1]],
                               "TN - validation": [tn_validation],
                               "TP - validation": [tp_validation],
                               "FN - validation": [fn_validation],
                               "FP - validation": [fp_validation],
                               "Accuracy - validation": [accuracy_validation],
                               "Specificity - validation": [specificity_validation],
                               "Precision - validation": [precision_validation],
                               "Recall - validation": [recall_validation],
                               "F1-score - validation": [f1_validation],
                               "MCC - validation": [mcc_validation],
                               "AUC - validation": [roc_auc_validation]})

    # Append the new record
    output_dataset = pd.concat([output_dataset, new_record], ignore_index=True, axis=0)

    # Export the dataset
    output_dataset.to_pickle(OUTPUT_PATH + "/ML_model_experiments.pkl")
    print("Esprimento aggiunto al dataset")

Provo a migliorare l'accuratezza del modello con la tecnica del cross-validation score

In [ ]:
num_split = 5
# prepare the cross-validation procedure
cv = KFold(n_splits=num_split, random_state=rnd_state, shuffle=True)
# create model
random_forest_classifier_cv = RandomForestClassifier(random_state=rnd_state, n_jobs=n_jobs)
# evaluate model
scores = cross_val_score(random_forest_classifier_cv, X_dataset, Y_dataset, scoring='f1', cv=cv, n_jobs=n_jobs)
# report performance
print("L'accuratezza con", num_split, "split è", np.mean(scores))

Testiamo diverse random forest con parametri diversi (hypertuning parameters)

In [ ]:
# Each list contains all the value of a specific hyperparameter we want to test
random_forest_hyp_n_estimators = [50, 100, 150, 200]
random_forest_hyp_max_features = ["auto", "sqrt", "log2"]
random_forest_hyp_criterions = ["entropy", "gini"]

# Create dictionary with each desired hyperparameter
random_forest_hyp_grid = {'n_estimators': random_forest_hyp_n_estimators,
                          'max_features': random_forest_hyp_max_features,
                          'criterion': random_forest_hyp_criterions}

# Use the random grid to search for best hyperparameters
rf_grid = GridSearchCV(estimator=RandomForestClassifier(),
                       param_grid=random_forest_hyp_grid,
                       cv=2,
                       scoring="f1",
                       n_jobs=4,
                       pre_dispatch="2*n_jobs",
                       verbose=4)
# Fit the random search model
rf_grid.fit(X_train, Y_train)

Stampo i parametri con lo score migliore

In [ ]:
print("Parameters:", rf_grid.best_params_)
print("Accuracy:", rf_grid.best_score_)

In [ ]:
grid_dataset = pd.DataFrame(rf_grid.cv_results_).sort_values(by=['mean_test_score'], ascending=False)

In [ ]:
grid_dataset.to_excel(OUTPUT_PATH + "/grid_search.xlsx")

In [ ]:
grid_dataset